In [4]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

from utils.text_analysis_transformers import RemovePunctuation, RemoveNonAscii
from utils.text_analysis_transformers import NltkWordTokenizer, WordLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from utils.TimeBasedCV import TimeBasedCV

from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

from sklearn.svm import SVR

from utils.data_utils import import_reddit_set, import_uk_confidence


from sklearn.feature_extraction.text import TfidfTransformer

from google.cloud import bigquery

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


# Reddit word count analysis

Aiming to create the table;

|             | Avg Intensity (weighted by the salience of the words in relation to the outcome variable) | Frequency of Words | Customer Confidence | Saving Ratio |
|-------------|---------------------------------------------------------------------------------------------------|----------------------------|-----------------------------|----------------------|
| Correlation |                                                                                                   |                            |                             |                      |
| 1 Month     |                                                                                                   |                            |                             |                      |
| 3 Month     |                                                                                                   |                            |                             |                      |
| 6 Month     |                                                                                                   |                            |                             |                      |
| 9 Months    |                                                                                                   |                            |                             |                      |

## Outputs;
Does each feature correlate with the target variables?
Do the targets correlate with each other? (sanity check)
Do the features have predictive power? 


In [5]:
client = bigquery.Client(location="US", project="goldenfleece")

query = """
    SELECT *
    FROM final_task.alltones
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    project='goldenfleece'
)  # API request - starts the query

all_tones_words = query_job.to_dataframe()

In [6]:
all_tones_words.head()

,words
0,ha
1,no
2,ass
3,axe
4,bad


In [36]:
pd.value_counts(all_tones_words['words'])

exposure       2
prisoners      2
probability    2
deadweight     2
contracting    2
              ..
yardstick      1
ticker         1
foreclosure    1
escapes        1
adopts         1
Name: words, Length: 7552, dtype: int64

## Interesting - not all values are unique. will drop duplicates.

In [7]:
unique_words = all_tones_words.drop_duplicates('words', ignore_index=True)

unique_words.head()

,words
0,ha
1,no
2,ass
3,axe
4,bad


In [46]:
pd.value_counts(unique_words['words'])

power           1
unbiased        1
manipulated     1
appeals         1
cruelty         1
               ..
remanded        1
legislator      1
defamations     1
invalidating    1
adopts          1
Name: words, Length: 7552, dtype: int64

In [8]:
unique_words.to_csv('data/vocabulary.csv')

In [49]:
unique_words_dict = unique_words['words'].to_dict()

unique_words_dict

{0: 'ha',
 1: 'no',
 2: 'ass',
 3: 'axe',
 4: 'bad',
 5: 'ban',
 6: 'big',
 7: 'cry',
 8: 'cut',
 9: 'die',
 10: 'dud',
 11: 'fad',
 12: 'fag',
 13: 'fan',
 14: 'fit',
 15: 'flu',
 16: 'ftw',
 17: 'fud',
 18: 'fun',
 19: 'gag',
 20: 'god',
 21: 'gun',
 22: 'hid',
 23: 'hug',
 24: 'ill',
 25: 'joy',
 26: 'lag',
 27: 'law',
 28: 'lie',
 29: 'lol',
 30: 'mad',
 31: 'may',
 32: 'odd',
 33: 'oks',
 34: 'pay',
 35: 'rig',
 36: 'rob',
 37: 'sad',
 38: 'shy',
 39: 'sue',
 40: 'top',
 41: 'war',
 42: 'win',
 43: 'won',
 44: 'woo',
 45: 'wow',
 46: 'wtf',
 47: 'yes',
 48: '144a',
 49: '1933',
 50: '1986',
 51: '2002',
 52: 'able',
 53: 'ache',
 54: 'acid',
 55: 'afdb',
 56: 'agog',
 57: 'alas',
 58: 'ante',
 59: 'anti',
 60: 'anxi',
 61: 'arab',
 62: 'area',
 63: 'arms',
 64: 'aval',
 65: 'avid',
 66: 'axed',
 67: 'back',
 68: 'bail',
 69: 'balk',
 70: 'bank',
 71: 'bans',
 72: 'base',
 73: 'bear',
 74: 'best',
 75: 'beta',
 76: 'bias',
 77: 'bill',
 78: 'blah',
 79: 'bloc',
 80: 'blue',
 81: 'b

In [50]:
# mapping needs to be reversed for the sklearn countvectorizer!
vocab = {v: k for k, v in unique_words_dict.items()}

In [10]:
reddit = import_reddit_set(rows=999999)
reddit['date'] = reddit['date'].dt.to_period("M")
reddit.set_index('date', inplace=True)

reddit.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1092: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,body
date,
2019-02,Exhibit A : Sir Christopher Cope : https://en....
2019-02,Am I supposed to feel bad for their choices or...
2019-02,&gt; which almost entirely seems to be an Isla...
2019-02,the darkest timeline
2019-02,A vote for Leave is a vote for Austerity 2: Br...


In [27]:
pd.unique(reddit.index)


PeriodIndex(['2019-02', '2019-03', '2019-12', '2019-06', '2019-08', '2019-11',
             '2019-09', '2019-04', '2019-01', '2019-07', '2019-05', '2019-10'],
            dtype='period[M]', name='date', freq='M')

In [28]:
months_of_text = []
index = []
for month in pd.unique(reddit.index):

    month_of_text = ''.join(reddit.loc[month, 'body'].values.tolist())

    # remove digits..
    month_of_text = ''.join(i for i in month_of_text if not i.isdigit())

    months_of_text.append(month_of_text)
    index.append(month)

In [51]:
word_frequency_pipe = Pipeline([
    ('remove_non_ascii', RemoveNonAscii()),
    ('remove_punctuation', RemovePunctuation()),
    ('lemmatize', WordLemmatizer()),
    ('count_vec', CountVectorizer(stop_words='english',
                                  lowercase=True,
                                  ngram_range=(1, 3), # this is very memory expensive!
                                  vocabulary= vocab)
                                    )
                                ])

word_tfidf_pipe = Pipeline([
    ('remove_non_ascii', RemoveNonAscii()),
    ('remove_punctuation', RemovePunctuation()),
    ('lemmatize', WordLemmatizer()),
    ('count_vec', CountVectorizer(stop_words='english',
                                  lowercase=True,
                                  vocabulary= vocab)),
     ('tfidf', TfidfTransformer()),
                                ])


In [53]:
matrix = word_frequency_pipe.fit_transform(months_of_text)
result = pd.DataFrame(matrix.toarray(), columns= word_frequency_pipe['count_vec'].get_feature_names(), index=index)
result.head(12)

,ha,no,ass,axe,bad,ban,big,cry,cut,die,...,misclassifications,misinterpretations,misrepresentations,multifunctionality,once-in-a-lifetime,oversimplification,unconstitutionally,extraterritoriality,mischaracterization,unconstitutionality
2019-02,84,0,53,19,1613,207,1070,0,340,273,...,0,0,1,0,0,7,0,0,1,0
2019-03,66,0,72,11,1867,281,1151,0,403,284,...,0,1,4,0,0,5,0,2,0,0
2019-12,90,0,89,19,2324,212,1635,0,582,429,...,0,0,2,0,0,9,1,0,0,0
2019-06,54,0,58,11,1646,204,944,0,442,266,...,0,1,2,0,0,10,1,0,1,0
2019-08,56,0,63,13,1763,227,1115,0,410,263,...,0,1,0,0,0,7,2,0,1,0
2019-11,59,0,53,11,1890,211,1284,0,578,265,...,0,2,2,0,0,6,0,0,0,0
2019-09,580,0,56,13,2128,275,1247,0,461,321,...,0,5,1,0,0,6,2,0,0,2
2019-04,75,0,60,9,1723,235,1057,0,345,267,...,0,0,3,0,0,11,0,0,3,0
2019-01,74,0,57,14,1852,188,1090,0,402,242,...,0,1,2,0,0,6,0,0,1,1
2019-07,117,0,68,10,1633,196,998,0,403,253,...,0,1,1,0,0,7,2,1,0,0


In [54]:
result.to_csv('data/reddit2019_tonal_wordcounts.csv')

In [57]:

def get_results(pipeline_transformer: Pipeline, text: list) -> pd.DataFrame:
    matrix = pipeline_transformer.fit_transform(text)
    
    return pd.DataFrame(matrix.toarray(), columns= pipeline_transformer['count_vec'].get_feature_names(), index=index)

#word_frequency_result = get_results(word_frequency_pipe, months_of_text)



In [58]:
tfidf_result = get_results(word_tfidf_pipe, months_of_text)
tfidf_result.head()

,ha,no,ass,axe,bad,ban,big,cry,cut,die,...,misclassifications,misinterpretations,misrepresentations,multifunctionality,once-in-a-lifetime,oversimplification,unconstitutionally,extraterritoriality,mischaracterization,unconstitutionality
2019-02,0.003199,0.0,0.002018,0.000724,0.061424,0.007883,0.040746,0.0,0.012947,0.010396,...,0.0,0.000000,0.000041,0.0,0.0,0.000267,0.000000,0.000000,0.000057,0.0
2019-03,0.001996,0.0,0.002178,0.000333,0.056476,0.008500,0.034817,0.0,0.012191,0.008591,...,0.0,0.000041,0.000131,0.0,0.0,0.000151,0.000000,0.000149,0.000000,0.0
2019-12,0.002219,0.0,0.002195,0.000469,0.057308,0.005228,0.040318,0.0,0.014352,0.010579,...,0.0,0.000000,0.000053,0.0,0.0,0.000222,0.000044,0.000000,0.000000,0.0
2019-06,0.002153,0.0,0.002312,0.000439,0.065627,0.008134,0.037638,0.0,0.017623,0.010606,...,0.0,0.000055,0.000086,0.0,0.0,0.000399,0.000071,0.000000,0.000059,0.0
2019-08,0.002044,0.0,0.002299,0.000474,0.064344,0.008285,0.040694,0.0,0.014964,0.009599,...,0.0,0.000050,0.000000,0.0,0.0,0.000255,0.000129,0.000000,0.000054,0.0


In [59]:
tfidf_result.to_csv('data/reddit2019_tonal_tfidf.csv')

## Outcome variable: 

In [60]:
# Import the outcome variable and preprocess
uk_confidence = import_uk_confidence()

uk_confidence.head()

,confidence
date,
2014-01,100.3960
2014-02,100.7097
2014-03,101.0020
2014-04,101.2360
2014-05,101.3725


In [78]:
y = uk_confidence.loc[tfidf_result.index, :]

In [89]:
# I'm not sure on how to correlate a matrix with against a vector, is this right? 
# Values don't look right!

correlation = np.correlate(tfidf_result.values.flatten(), y.values.flatten())
correlation[:10]

array([13.93133883, 13.63588972, 14.18392673, 14.72960666, 14.69030981,
        8.53976176,  7.78326387,  4.23331689,  4.23441684,  4.07295309])

In [90]:
def replace_quarterly(row):
    if row["Title"].endswith("Q1"):
        return row["Title"].replace(" Q1", "-01-01")
    elif row["Title"].endswith("Q2"):
        return row["Title"].replace(" Q2", "-04-01")
    elif row["Title"].endswith("Q3"):
        return row["Title"].replace(" Q3", "-07-01")
    elif row["Title"].endswith("Q4"):
        return row["Title"].replace(" Q4", "-10-01")

def import_household_savings() -> pd.DataFrame:
    """
    Returns UK household savings ratios
    """
    household_savings_df = pd.read_csv("data/household_savings_ratio.csv")
    household_savings_df["quarterly_data"] = [
        True if "Q" in x else False for x in household_savings_df["Title"]
    ]
    household_savings_df = household_savings_df[
        household_savings_df["quarterly_data"] == True
    ]
    household_savings_df["date"] = pd.to_datetime(
        household_savings_df.apply(replace_quarterly, axis=1)
    ).dt.to_period("M")
    household_savings_df = household_savings_df.set_index("date").resample("M").ffill()
    household_savings_df = household_savings_df.rename(
        {
            "Households (S.14): Households' saving ratio (per cent): Current price: £m: SA": "savings_ratio"
        },
        axis=1,
    )
    return household_savings_df[["savings_ratio"]]

In [ ]:
household_savings.head()

In [ ]:
def import_uk_confidence():
    all_confidence = pd.read_csv('data/consumer_confidence_index.csv',
                               usecols=['TIME', 'Value', 'LOCATION'])

    uk_confidence = all_confidence.loc[all_confidence.LOCATION == "GBR"]

    assert all(pd.value_counts(uk_confidence.TIME) == 1), "duplicate entries for the same time period"

    date = pd.to_datetime(uk_confidence.TIME, format="%Y-%m")

    # clean dataframe:
    df = pd.DataFrame({'date': date, 'confidence': uk_confidence.Value})
    df['date'] = df['date'].dt.to_period("M")
    df = df.set_index(['date'], drop=True)

    return df

In [ ]:
pd.concat([household_savings, uk_confidence], axis=1).tail(10)

In [ ]:
target_df = pd.concat([household_savings, uk_confidence], axis=1).dropna()